In [1]:
import numpy as np
import pandas as pd
import gc
import pickle
import time

import matplotlib.pyplot as plt
%matplotlib inline

from xgboost import XGBRegressor
from xgboost import plot_importance

def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)

ImportError: No module named 'xgboost'

In [ ]:
data = pd.read_pickle('matrix.pickle')
X_train = data[data.date_block_num < 33].drop(['date_block_num','item_cnt_month'], axis=1)
Y_train = data[data.date_block_num < 33]['item_cnt_month']
X_valid = data[data.date_block_num == 33].drop(['date_block_num','item_cnt_month'], axis=1)
Y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['date_block_num','item_cnt_month'], axis=1)
del data
gc.collect();

In [ ]:
ts = time.time()

m1 = XGBRegressor(
    max_depth=15,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    colsample_bylevel=0.8, 
    subsample=0.8, 
    eta=0.3, 
    num_round=1000,
    seed=1, 
    nthread=24)

m1.fit(
    X_train, 
    Y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
    verbose=True, 
    early_stopping_rounds = 10)

time.time() - ts

In [ ]:
#pickle.dump(m1, open("m1.pickle.dat", "wb"))
#m1 = pickle.load(open("m1.pickle.dat", "rb"))

In [ ]:
plot_features(m1, (18,42))

In [ ]:
Y_test = m1.predict(X_test).clip(0, 20)

test = pd.read_csv('test.csv.gz', compression='gzip').set_index('ID')
submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test
})
submission.to_csv('submission.csv', index=False)